In [4]:
import pandas as pd

In [5]:
# Reading csv
df = pd.read_csv("reply_classification_dataset.csv")
df.head(10)

,reply,label
0,Can we discuss pricing??,NEUTRAL
1,"Im excited to explore this further, plz send c...",POSITIVE
2,We not looking for new solutions.,negative
3,Could u clarify features included?,neutral
4,"lets,, schedule a meeting to dive deeper",positive
5,Please remove me from list,NEGATIVE
6,"This looks promising, send specs!!",Positive
7,Ill need to check w/ my team,Neutral
8,Were alredy using similar product,negative
9,Looking forward to demo!,POSITIVE


In [6]:
# drop rows with missing values
df.dropna(subset=['reply', 'label'], inplace=True)

print("\nDataFrame after handling missing values:")
print(df.info())


DataFrame after handling missing values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   reply   2129 non-null   object
 1   label   2129 non-null   object
dtypes: object(2)
memory usage: 33.4+ KB
None


In [8]:
import re

def clean_text(text):
    text = str(text).lower()

    #[^a-z\s] character that is NOT a lowercase letter or a whitespace character.
    text = re.sub(r'[^a-z\s]', '', text)

    #Remove extra whitespace
    text = text.strip()
    text = re.sub(r'\s+', ' ', text)
    return text


df['reply'] = df['reply'].apply(clean_text)
df['label'] = df['label'].str.lower()

In [11]:
# Display the cleaned dataframe
print("\nFinal DataFrame:")
print(df.head(10))

print("\nlabels:")
print(df['label'].unique())


Final DataFrame:
                                               reply     label
0                             can we discuss pricing   neutral
1  im excited to explore this further plz send co...  positive
2                   we not looking for new solutions  negative
3                  could u clarify features included   neutral
4             lets schedule a meeting to dive deeper  positive
5                         please remove me from list  negative
6                    this looks promising send specs  positive
7                        ill need to check w my team   neutral
8                  were alredy using similar product  negative
9                            looking forward to demo  positive

labels:
['neutral' 'positive' 'negative']


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X = df['reply']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [25]:
#removing duplicate rows
initial_rows = len(df)
df.drop_duplicates(subset=['reply', 'label'], inplace=True)
final_rows = len(df)

print(f"Removed {initial_rows - final_rows} duplicate rows.")
print(f"Remaining rows in the dataset: {final_rows}")

Removed 0 duplicate rows.
Remaining rows in the dataset: 317


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline

#Logistic Regression
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [22]:
#predictions
y_pred_logistic = logistic_model.predict(X_test_tfidf)
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
f1_logistic = f1_score(y_test, y_pred_logistic, average='weighted')

print(f"Logistic Regression Model Accuracy: {accuracy_logistic:.4f}")
print(f"Logistic Regression Model F1 Score: {f1_logistic:.4f}")

Logistic Regression Model Accuracy: 1.0000
Logistic Regression Model F1 Score: 1.0000


In [23]:
def predict_sentiment(text):
    preprocessed_text = clean_text(text)
    text_tfidf = tfidf_vectorizer.transform([preprocessed_text])
    prediction = logistic_model.predict(text_tfidf)
    return prediction[0]

# Assuming your clean_text() function and your models are in the environmen

MODEL TESTING


In [24]:
# Example 1:
reply1 = "This looks great, I'm ready to move forward."
print(f"Reply: '{reply1}' -> Predicted Label: {predict_sentiment(reply1)}")

# Example 2:
reply2 = "Thanks for your email, but we're not interested at this time."
print(f"Reply: '{reply2}' -> Predicted Label: {predict_sentiment(reply2)}")

# Example 3:
reply3 = "Could you send me some more details about your pricing plans?"
print(f"Reply: '{reply3}' -> Predicted Label: {predict_sentiment(reply3)}")

# Example 4:
reply4 = "The demo was good, but I need to consult with my team."
print(f"Reply: '{reply4}' -> Predicted Label: {predict_sentiment(reply4)}")

Reply: 'This looks great, I'm ready to move forward.' -> Predicted Label: positive
Reply: 'Thanks for your email, but we're not interested at this time.' -> Predicted Label: negative
Reply: 'Could you send me some more details about your pricing plans?' -> Predicted Label: neutral
Reply: 'The demo was good, but I need to consult with my team.' -> Predicted Label: positive


In [26]:
!pip install transformers datasets accelerate
!pip install torch

In [28]:
import pandas as pd
from datasets import Dataset, ClassLabel, Features, Value

label_mapping = {"positive": 0, "negative": 1, "neutral": 2}
df["label_int"] = df["label"].map(label_mapping)

#Hugging Face Dataset
dataset = Dataset.from_pandas(df)

dataset = dataset.cast_column(
    "label_int", ClassLabel(names=["positive", "negative", "neutral"])
)

dataset = dataset.rename_column("reply", "text")
dataset = dataset.remove_columns(["label"])
dataset = dataset.rename_column("label_int", "label")

Casting the dataset:   0%|          | 0/317 [00:00<?, ? examples/s]

In [29]:
if '__index_level_0__' in dataset.column_names:
    dataset = dataset.remove_columns(["__index_level_0__"])

print(dataset.column_names)

['text', 'label']


In [30]:
# Split the dataset
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 253
Test dataset size: 64


In [31]:
from transformers import AutoTokenizer

#DistilBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text"])
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["text"])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/253 [00:00<?, ? examples/s]

Map:   0%|          | 0/64 [00:00<?, ? examples/s]

In [34]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate accuracy and f1 score
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')

    return {"accuracy": accuracy, "f1": f1}

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
# Configure training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none",
)

In [37]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics,
)


trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.889721,0.765625,0.758900
2,No log,0.598736,0.953125,0.954044
3,No log,0.505215,0.953125,0.954044


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=48, training_loss=0.7716845671335856, metrics={'train_runtime': 2638.0024, 'train_samples_per_second': 0.288, 'train_steps_per_second': 0.018, 'total_flos': 100544548617216.0, 'train_loss': 0.7716845671335856, 'epoch': 3.0})

In [38]:
# Evaluate the fine-tuned model on the test set
transformer_results = trainer.evaluate()

# Print the results
print("Transformer Model Evaluation Results:")
print(f"Accuracy: {transformer_results['eval_accuracy']:.4f}")
print(f"F1 Score: {transformer_results['eval_f1']:.4f}")

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Transformer Model Evaluation Results:
Accuracy: 0.9531
F1 Score: 0.9540
